In [104]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional

In [105]:
import importlib

import model
importlib.reload(model)

<module 'model' from 'c:\\Users\\Site Oracle\\reduceComplexity\\model.py'>

In [106]:
aapl = model.Company("Apple", "AAPL", model.SECFilingRepository())

In [107]:
aapl.cik

'0000320193'

In [108]:
print(aapl.filings)

getting filings
filings retrieved
returning filings
[<model.Filing object at 0x000001971EEC7250>, <model.Filing object at 0x000001971EF01490>, <model.Filing object at 0x000001972126E590>, <model.Filing object at 0x000001972126EE10>, <model.Filing object at 0x000001972126ECD0>, <model.Filing object at 0x000001972126EBD0>, <model.Filing object at 0x000001972126EC50>, <model.Filing object at 0x000001972126DF10>, <model.Filing object at 0x000001972126DF50>, <model.Filing object at 0x000001972126ED50>, <model.Filing object at 0x000001972126DED0>, <model.Filing object at 0x000001972126E010>, <model.Filing object at 0x000001972126E090>, <model.Filing object at 0x000001972126E110>, <model.Filing object at 0x000001972126E050>, <model.Filing object at 0x000001972126DE90>, <model.Filing object at 0x000001972126DFD0>, <model.Filing object at 0x000001972126DD10>, <model.Filing object at 0x000001972126E410>, <model.Filing object at 0x000001972126DB90>, <model.Filing object at 0x000001972126D950>, <m

In [109]:
def get_annual_filing(aapl):
    for filing in aapl.filings:
        if filing.form == "10-K":
            return filing
        

In [110]:
filing = get_annual_filing(aapl)

returning filings


In [111]:
print(filing.cik)
print(filing.accession_number)
print(filing.primary_document)


0000320193
0000320193-24-000123
aapl-20240928.htm


In [ ]:
# import json
# import os
# from datetime import datetime

# # Get the filing data
# filing_data = filing.data

# # Create a filename with company ticker, form type, and filing date
# filename = f"AAPL_10Q_{filing.filing_date.replace('-', '')}_data.json"

# # Save the filing data to a JSON file
# with open(filename, 'w') as f:
#     json.dump(filing_data, f, indent=4)

# print(f"Filing data saved to {filename}")


Filing data saved to AAPL_10Q_20241101_data.json


In [113]:
df = filing.income_statement.table

In [114]:
df.columns

Index(['2021-09-26:2022-09-24', '2022-09-25:2023-09-30',
       '2023-10-01:2024-09-28'],
      dtype='object', name='date_range')

In [115]:
df

date_range,2021-09-26:2022-09-24,2022-09-25:2023-09-30,2023-10-01:2024-09-28
metric,,,
CostOfGoodsAndServicesSold,2.235460e+11,2.141370e+11,2.103520e+11
EarningsPerShareBasic,6.150000e+00,6.160000e+00,6.110000e+00
EarningsPerShareDiluted,6.110000e+00,6.130000e+00,6.080000e+00
GrossProfit,1.707820e+11,1.691480e+11,1.806830e+11
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,1.191030e+11,1.137360e+11,1.234850e+11
IncomeTaxExpenseBenefit,1.930000e+10,1.674100e+10,2.974900e+10
NetIncomeLoss,9.980300e+10,9.699500e+10,9.373600e+10
NonoperatingIncomeExpense,-3.340000e+08,-5.650000e+08,2.690000e+08
OperatingExpenses,5.134500e+10,5.484700e+10,5.746700e+10
